In [28]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from fastai.data.transforms import get_image_files, get_files
from fastai.vision.data import SegmentationDataLoaders
import os
from os import path
from fastseg.image import colorize, blend
from PIL import Image
from tqdm import tqdm
import albumentations as A
import math

In [49]:
base_dir = "/media/nevin/Trash Games1/culane/driver_23_30frame"
processed_dir = "/media/nevin/Trash Games1/culane/augmented"

dirs = os.listdir(base_dir)

output = []

for dir in dirs:
    dir = path.join(base_dir, dir)
    images = get_image_files(dir)
    step = math.floor(len(images) / 14)
    if step == 0:
        continue
    for i in range(0, len(images), step):
        output.append(images[i])

# print(len(output))
print(len(output))

5284


In [73]:
def get_points(label_file):
    with open(label_file) as file:
        lines = file.readlines()
        lines = [line.rstrip() for line in lines]

    points = [[] for line in lines]
    for idx, line in enumerate(lines):
        split = line.split(" ")
        for i in range(0, len(split), 2):
            points[idx].append([int(float(split[i])), int(float(split[i + 1]))]) # it works
    
    return np.array(points, dtype=object)


def gen_seg(image, points):
    h, w, c = image.shape
    seg_image = np.zeros((h, w), np.uint8)
    for i in range(len(points)):
        point = np.asarray(points[i])
        x = point[:, 0]
        y = point[:, 1]

        draw_points = (np.asarray([x, y]).T).astype(np.int32)

        seg_image = cv.polylines(seg_image, [draw_points], False, color=(1, 0, 0), thickness=10)
    
    return seg_image


for image in tqdm(output):
    img_save_path = os.path.join(processed_dir, "images/", image.name)
    seg_save_path = os.path.join(processed_dir, "labels/", f"{image.stem}.png")
    label_path = f"{image.parent}/{image.stem}.lines.txt"

    img = cv.imread(str(image))
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

    points = get_points(label_path)
    seg_mask = gen_seg(img, points)

    # maybe don't do this for data augmentation later but for now it's fine
    img = img[0:500, 400:1200]
    seg_mask = seg_mask[0:500, 400:1200]

    img = Image.fromarray(img)
    img.save(img_save_path)

    seg_mask = Image.fromarray(seg_mask)
    seg_mask.save(seg_save_path)

  0%|          | 0/5284 [00:00<?, ?it/s]


In [61]:
def label_func(input):
    return f"{input.parent}/{input.stem}.lines.txt"

dls = SegmentationDataLoaders


/media/nevin/Trash Games1/culane/driver_23_30frame/05151640_0419.MP4/00000.lines.txt
